# NPD Field data analysis and visualisation
## https://factpages.npd.no/en/field/tableview/overview
This notebook contains analysis and visualisations of the data available for export under the 'Field' tab at the link above.

The process is to specify a Company licensee and display the Fields they have an interest in. From this, the in place volumes and reserves of those fields, with totals, can be extracted and displayed.

There is also visualisations on number of Licensees and the number of fields they have an interest in, number of producing fields, breakdown of fields by area (North Sea, Norwegian Sea and Barents Sea).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_theme()
pd.options.display.max_rows = 500

In [2]:
# This list represents the data under the Field tab (plus one under the Company tab).
# The list items are used as keys to the links which are held in the load package.

fields = ['field_overview',
          'field_status',
          'field_operators',
          'field_owners',
          'field_licensees',
          'field_reserves',
          'field_inplaceVol',
          'field_description',
          'comp_reserves',
         ]

In [3]:
from load import load, calculate

# Field Licensees

In [4]:
licensees = load(fields[4])

In [5]:
# Current licences don't have a date in 'fldLicenseeTo' field.
curr_lic = licensees[licensees.fldLicenseeTo.isnull()]
data = curr_lic.cmpLongName.value_counts()

## Current Field Licensee Companies

In [6]:
# Select a company by the integer value to be used as input to the Company specific visualisations
licence_companies = list(enumerate(data.index, start=1))
print(licence_companies)
select = int(input("Select a company by associate number: "))
company = licence_companies[select-1][1]
print("*"*52)
print(f"Selected company is {company}")

[(1, 'Equinor Energy AS'), (2, 'Petoro AS'), (3, 'Vår Energi AS'), (4, 'Total E&P Norge AS'), (5, 'Wintershall Dea Norge AS'), (6, 'Aker BP ASA'), (7, 'ConocoPhillips Skandinavia AS'), (8, 'Neptune Energy Norge AS'), (9, 'Spirit Energy Norway AS'), (10, 'DNO Norge AS'), (11, 'LOTOS Exploration and Production Norge AS'), (12, 'Repsol Norge AS'), (13, 'Idemitsu Petroleum Norge AS'), (14, 'PGNiG Upstream Norway AS'), (15, 'Lundin Energy Norway AS'), (16, 'KUFPEC Norway AS'), (17, 'A/S Norske Shell'), (18, 'OMV (Norge) AS'), (19, 'Wellesley Petroleum AS'), (20, 'OKEA ASA'), (21, 'INEOS E&P Norge AS'), (22, 'Suncor Energy Norge AS'), (23, 'DNO North Sea (ROGB) Limited'), (24, 'Petrolia NOCO AS'), (25, 'CapeOmega AS'), (26, 'Repsol Sinopec North Sea Limited'), (27, 'Repsol Sinopec Resources UK Limited'), (28, 'Mime Petroleum AS'), (29, 'Dana Petroleum (BVUK) Limited'), (30, 'Sval Energi AS'), (31, 'Pandion Energy AS'), (32, 'Edison Norge AS'), (33, 'Chrysaor Norge AS'), (34, 'ONE-Dyas Norge 

In [7]:
# Format label strings to only show first word in Company long name
labels = data.index
labelf = [f'{label.split()[0]}' for label in labels]

## Plot a list of all companies that have a current interest is Fields on the NCS

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6), tight_layout=True)
data.plot(kind='bar')
ax.xaxis.set_ticklabels(labelf)
plt.xticks(rotation=90); plt.title('Field Licensees and number of fields', fontsize=22)
plt.gca().get_xticklabels()[select-1].set_color('red')
plt.savefig('Field_licensees.png')
plt.show()

In [ ]:
company_licences = licensees[licensees['cmpLongName']==company]
company_licences = company_licences.loc[company_licences.groupby('fldName').fldLicenseeFrom.idxmax(),:]

## Print the number of Fields that the selected company has an interest in

In [ ]:
print(f"Number of Fields for {company}: {len(company_licences)}")

In [ ]:
company_fields = company_licences['fldName']
company_fields

In [ ]:
inplacevols = load(fields[6])

# In place volumes for specified Company

In [ ]:
company_inplacevols = calculate(company_fields, inplacevols)
company_inplacevols.to_csv('in_place_volumes.csv', index=False);

In [ ]:
company_inplacevols

In [ ]:
# Field reserves
reserves = load(fields[5])
# Field reserves at Company share ('fldCompanyShare')
reserves_comp = load(fields[-1]) 

# Reserves for each Field the specified company has an interest in

In [ ]:
reserves_comp = reserves_comp[reserves_comp['cmpLongName']==company]
reserves_comp.drop(['cmpLongName', 'cmpDateOffResEstDisplay', 'fldNpdidField', 'cmpNpdidCompany'], axis=1, inplace=True)
reserves_comp.loc['Total'] = reserves_comp.sum(numeric_only=True, axis=0)
reserves_comp.to_csv('reserves_company.csv', index=False)
reserves_comp

## Display total reserves for each field - not company share.

In [ ]:
company_reserves = calculate(company_fields, reserves)
company_reserves.to_csv('reserves_all.csv', index=False)
company_reserves

# Overview

In [ ]:
overview = load(fields[0])

## Breakdown of all fields on the NCS

In [ ]:
overview.groupby('fldCurrentActivitySatus').size().plot(kind='bar', figsize=(10,6), fontsize=18)
plt.savefig('All Fields.png'); plt.show()

In [ ]:
producing_fields = overview[overview['fldCurrentActivitySatus']=='Producing']['cmpLongName'].value_counts()
producing_approved_fields = overview[overview['fldCurrentActivitySatus'].isin(['Approved for production'])]['cmpLongName'].value_counts()
shutdown_fields = overview[overview['fldCurrentActivitySatus']=="Shut down"]['cmpLongName'].value_counts()

print(f"Number of fields: {overview.shape[0]}")
print(f"Number of producing fields: {producing_fields.sum()}")
print(f"Number of producing and approved fields: {producing_approved_fields.sum()}")
print(f"Number of shut down fields: {shutdown_fields.sum()}") # doesn't include 3 NaN entries

## Status of Fields by Area

In [ ]:
data = overview.groupby(['fldMainArea', 'fldCurrentActivitySatus']).size().unstack()
data.loc['Column Total'] = data.sum(numeric_only=True, axis=0)
print(data)
data.plot(kind='bar', figsize=(15,6), fontsize=22)
plt.savefig('Fields by Area.png'); plt.show()

# Backup

# Status

In [ ]:
status = load(fields[1])

In [ ]:
# status.head()

In [ ]:
status.dropna(inplace=True)
status = status.loc[status.groupby('fldName').fldStatusToDate.idxmax(),:]

In [ ]:
status['fldStatus'].value_counts()
#  the headings and values look transposed

In [ ]:
status['fldName'].nunique()

# Operators

In [ ]:
operators = load(fields[2])

In [ ]:
print(f" Number of Operators: {operators['cmpLongName'].nunique()}")
print(f" Number of Fields: {operators['fldName'].nunique()}")

In [ ]:
# Current Operators don't have a date in the 'fldOperatorTo' field
operators[operators['fldOperatorTo'].isnull()]['cmpLongName'].value_counts() # 121 - doesn't include NaN

# Description

In [ ]:
description = load(fields[7])

In [ ]:
description.head()